# Assessing Trading Algorithms

In [1]:
# Import required libraries
import pandas as pd
import warnings 
import numpy as np
from pathlib import Path
import hvplot.pandas
warnings.filterwarnings('ignore')

In [2]:
# Read the vnq.csv file from the Resources folder into a Pandas DataFrame
# Set the `Date` column as the DateTimeIndex
fntk_df = pd.read_csv(
    Path("../Resources/fntk2.csv"), 
    index_col="Date",
    parse_dates=True, 
    infer_datetime_format=True
)

# Review the DataFrame
fntk_df.head()

,Close
Date,
2020-01-02 00:00:00-05:00,209.7800
2020-01-03 00:00:00-05:00,208.5100
2020-01-06 00:00:00-05:00,212.5855
2020-01-07 00:00:00-05:00,213.0500
2020-01-08 00:00:00-05:00,215.3000


In [3]:
# Create a DataFrame filtering only the index and Close columns
fntk_df = fntk_df.loc[:,["Close"]]

# Review the DataFrame
fntk_df.head()

,Close
Date,
2020-01-02 00:00:00-05:00,209.7800
2020-01-03 00:00:00-05:00,208.5100
2020-01-06 00:00:00-05:00,212.5855
2020-01-07 00:00:00-05:00,213.0500
2020-01-08 00:00:00-05:00,215.3000


In [4]:
# Use hvplot to visualize the DataFrame
fntk_df.hvplot()

:Curve   [Date]   (Close)

In [5]:
# Use hvplot to visualize the DataFrame
fntk_df['daily_returns'] = fntk_df['Close'].pct_change()
fntk_df['daily_returns_lagged'] = fntk_df['daily_returns'].shift(1)
fntk_df.head()

,Close,daily_returns,daily_returns_lagged
Date,,,
2020-01-02 00:00:00-05:00,209.7800,NaN,NaN
2020-01-03 00:00:00-05:00,208.5100,-0.006054,NaN
2020-01-06 00:00:00-05:00,212.5855,0.019546,-0.006054
2020-01-07 00:00:00-05:00,213.0500,0.002185,0.019546
2020-01-08 00:00:00-05:00,215.3000,0.010561,0.002185


In [6]:
# Use hvplot to visualize the DataFrame
fntk_df['daily_returns'] = fntk_df['Close'].pct_change()
fntk_df['daily_returns_lagged'] = fntk_df['daily_returns'].shift(1)
fntk_df.head()# Use hvplot to visualize the DataFrame

,Close,daily_returns,daily_returns_lagged
Date,,,
2020-01-02 00:00:00-05:00,209.7800,NaN,NaN
2020-01-03 00:00:00-05:00,208.5100,-0.006054,NaN
2020-01-06 00:00:00-05:00,212.5855,0.019546,-0.006054
2020-01-07 00:00:00-05:00,213.0500,0.002185,0.019546
2020-01-08 00:00:00-05:00,215.3000,0.010561,0.002185


In [7]:
fntk_df[['daily_returns','daily_returns_lagged']].corr().style.background_gradient()

,daily_returns,daily_returns_lagged
daily_returns,1.000000,-0.320369
daily_returns_lagged,-0.320369,1.000000


In [8]:
fntk_df['trade_type'] =np.where(fntk_df['daily_returns_lagged']<0,1,-1)

In [9]:
# Review the DataFrame
fntk_df.head(10)

,Close,daily_returns,daily_returns_lagged,trade_type
Date,,,,
2020-01-02 00:00:00-05:00,209.7800,NaN,NaN,-1
2020-01-03 00:00:00-05:00,208.5100,-0.006054,NaN,-1
2020-01-06 00:00:00-05:00,212.5855,0.019546,-0.006054,1
2020-01-07 00:00:00-05:00,213.0500,0.002185,0.019546,-1
2020-01-08 00:00:00-05:00,215.3000,0.010561,0.002185,-1
2020-01-09 00:00:00-05:00,218.3100,0.013980,0.010561,-1
2020-01-10 00:00:00-05:00,218.0700,-0.001099,0.013980,-1
2020-01-13 00:00:00-05:00,221.9600,0.017838,-0.001099,1
2020-01-14 00:00:00-05:00,219.0600,-0.013065,0.017838,-1


In [10]:
fntk_df.dropna(inplace=True)

## Calculate the Cumulative Return for the Trading Period

In [11]:
fntk_df['algorithm_returns']= fntk_df['daily_returns'] * fntk_df['trade_type']

In [12]:
fntk_df['algorithm_cumulative_returns']=(fntk_df['algorithm_returns'] + 1).cumprod()

In [13]:
fntk_df['algorithm_cumulative_returns'].hvplot()

:Curve   [Date]   (algorithm_cumulative_returns)

## Calculate the Cumulative Return for the Trading Period

In [14]:
total_return = round((fntk_df['algorithm_cumulative_returns'][-1] - 1) * 100, 2)

# Print the profit/loss metrics
print(f"The total_return of the trading strategy is {total_return}%.")

The total_return of the trading strategy is 56.33%.


## Calculate the Cumulative Return for a 'Buy and Hold' Strategy

In [15]:
fntk_df['buy_hold_cumulative_returns']=(fntk_df['daily_returns'] + 1).cumprod()

In [16]:
fntk_df['buy_hold_cumulative_returns'].hvplot() * fntk_df['algorithm_cumulative_returns'].hvplot()

:Overlay
   .Curve.Buy_hold_cumulative_returns  :Curve   [Date]   (buy_hold_cumulative_returns)
   .Curve.Algorithm_cumulative_returns :Curve   [Date]   (algorithm_cumulative_returns)

In [17]:
total_return = round((fntk_df['buy_hold_cumulative_returns'][-1] - 1) * 100, 2)

print(f"The total_return of the trading strategy is {total_return}%.")

The total_return of the trading strategy is 10.96%.
